# Phi-4 Mini ONNX 병렬 함수 호출 튜토리얼

이 노트북은 Phi-4 Mini와 ONNX Runtime GenAI를 사용하여 병렬 함수 호출을 수행하는 방법을 보여줍니다. 함수 호출은 사용자의 요청에 따라 모델이 외부 도구와 API를 지능적으로 호출할 수 있도록 합니다.

## 개요

이 튜토리얼에서는 다음을 배우게 됩니다:
- Phi-4 Mini를 ONNX Runtime GenAI와 함께 설정하는 방법
- 항공편 및 호텔 예약을 위한 함수 스키마 정의
- 구조화된 출력을 위한 Lark 문법을 사용한 가이드 생성
- 복잡한 여행 예약 시나리오를 위한 병렬 함수 호출 실행

## 사전 준비 사항

이 노트북을 실행하기 전에 다음을 확인하세요:
- Phi-4 Mini ONNX 모델을 다운로드했는지
- `onnxruntime-genai` 패키지를 설치했는지
- 함수 호출 개념에 대한 기본적인 이해


## 1단계: 필요한 라이브러리 가져오기

먼저, 함수 호출 구현에 필요한 라이브러리를 가져옵니다.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## 2단계: 모델 설정 및 구성

이제 Phi-4 Mini ONNX 모델을 구성하겠습니다. 실제 모델 디렉토리 경로로 교체해야 합니다.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## 3단계: 생성 매개변수 구성

모델의 출력 동작을 제어하기 위해 생성 매개변수를 설정합니다. 이러한 설정은 함수 호출에 대해 결정적이고 집중된 응답을 보장합니다.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Step 4: 사용 가능한 함수 정의

여기서는 AI 어시스턴트가 호출할 수 있는 함수들을 정의합니다. 이 예시에서는 여행 관련 두 가지 함수가 있습니다:
1. **booking_flight_tickets**: 공항 간 항공편 예약을 위한 함수
2. **booking_hotels**: 호텔 숙박 예약을 위한 함수

함수 정의는 OpenAI의 함수 호출 스키마 형식을 따릅니다.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## 5단계: 문법 생성 보조 함수

이 보조 함수들은 함수 정의를 Lark 문법 형식으로 변환하여 ONNX Runtime GenAI가 안내된 생성을 수행할 수 있도록 합니다. 이를 통해 모델이 올바른 JSON 형식으로 유효한 함수 호출을 출력하도록 보장합니다.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## 6단계: 문법 생성 테스트

문법 생성 함수가 도구 정의를 올바른 형식으로 변환하는지 테스트해 봅시다.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## 7단계: 시스템 프롬프트 및 생성기 준비

이제 모델에게 사용 가능한 도구를 알려주는 시스템 프롬프트를 만들고, 안내된 생성 매개변수로 생성기를 설정하겠습니다.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## 8단계: 안내 생성으로 생성기 초기화하기

이제 설정된 매개변수를 사용하여 생성기를 만들고, 안내 생성을 위해 Lark 문법을 적용하겠습니다.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## 9단계: 병렬 함수 호출 테스트

이제 여러 함수를 호출해야 하는 복잡한 여행 예약 시나리오로 설정을 테스트해봅시다.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### 함수 호출 생성

모델은 이제 사용자 요청에 따라 구조화된 함수 호출을 생성합니다. 가이드된 생성 덕분에 출력은 바로 실행 가능한 유효한 JSON 형식으로 제공됩니다.

**예상 출력**: 모델은 다음에 대한 함수 호출을 생성해야 합니다:
1. `booking_flight_tickets` - 베이징(PEK)에서 파리(CDG)로 가는 항공편 세부 정보
2. `booking_hotels` - 파리 숙박 세부 정보

아래 셀을 실행하여 실시간 생성을 확인하세요:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## 결론

🎉 **축하합니다!** ONNX Runtime GenAI를 사용하여 Phi-4 Mini로 병렬 함수 호출을 성공적으로 구현하셨습니다.

### 배운 내용:

1. **모델 설정**: ONNX Runtime GenAI로 Phi-4 Mini를 설정하는 방법
2. **함수 정의**: AI 함수 호출을 위한 함수 스키마를 정의하는 방법
3. **유도된 생성**: 구조화된 출력 생성을 위해 Lark 문법을 사용하는 방법
4. **병렬 함수 호출**: 여러 함수 호출이 필요한 복잡한 요청을 처리하는 방법

### 주요 혜택:

- ✅ **구조화된 출력**: 유도된 생성은 유효한 JSON 함수 호출을 보장합니다
- ✅ **병렬 처리**: 단일 요청에서 여러 함수 호출을 처리합니다
- ✅ **고성능**: ONNX Runtime은 최적화된 추론을 제공합니다
- ✅ **유연한 스키마**: 함수 정의를 쉽게 추가하거나 수정할 수 있습니다

### 참고 자료:

- [Phi-4 Mini 문서](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI 문서](https://onnxruntime.ai/docs/genai/)
- [함수 호출 모범 사례](https://platform.openai.com/docs/guides/function-calling)



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있지만, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 신뢰할 수 있는 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
